In [ ]:
import torch
from tqdm import tqdm
from transformers import logging
logging.set_verbosity_error()
from transformers import BertTokenizer
import os
from pathlib import Path

from utils.datasets import DatasetWordPiece
from models.tvae_trainer import TVAETrainer
from models.tvae_model import TVAE

print(torch.cuda.is_available())
print(torch.__version__)

# Switch to correct folder
os.chdir("..")

In [ ]:
s = 

model= "deepset/gbert-base"
tokenizer= BertTokenizer.from_pretrained(model)

encoding = tokenizer.encode(s)
tokens = tokenizer.tokenize(s)
print(encoding)
print(tokens)

In [ ]:
settings = {
    "save/2023-01-18_Pin_down_Lr/checkpoints/2023-01-20_20:44:29_TVAE_German/2023-01-20_20:44:29_TVAE_RegTrue/model.pt": [
        ["Simple","Das bedeutet: Er hat sehr viele schlimme Verletzungen."]
    ]
}

In [ ]:
dataset = DatasetWordPiece(large=False, max_length=128)
trainer = TVAETrainer(dataset=dataset, model=model)
model = TVAE(ntoken=dataset.vocab_size)
model.cuda()

# needed for processing batch data
fake_label = torch.IntTensor([[1]])

for model, sents in settings.items():
    model.load_state_dict(torch.load(path_model))
    for sent in sents:
        t = torch.Tensor(dataset.encode(sent))
        t = t.view(1, -1).long()
        batch = trainer.process_batch_data((t, fake_label))

        d = {}
        for i, k in enumerate(["src", "tgt", "tgt_true", "tgt_mask", "memory_mask", "src_key_padding_mask", "tgt_key_padding_mask", "labels"]):
            d[k] = batch[i]

        output = model(
            src=d["src"],
            tgt=d["tgt"],
            tgt_mask=d["tgt_mask"],
            memory_mask=d["memory_mask"],
            src_key_padding_mask=d["src_key_padding_mask"],
            tgt_key_padding_mask=d["tgt_key_padding_mask"]
        )

        prob = output[0]
        out_tokens = torch.argmax(prob, dim=-1)
        out_tokens = [int(i) for i in list(out_tokens.data.to("cpu")[0])]

        print(dataset.tokenizer.decode(out_tokens))
        print(acc(prob, d["tgt_true"]))
        print(list(d["tgt_true"][0].cpu().numpy()))
        print(out_tokens)


In [ ]:
def compute_latent_interpolations(self, latent_code, dim=0, num_points=10):
    x = torch.linspace(-4.0, 4.0, num_points)
    z = to_cuda_variable(torch.from_numpy(latent_code))
    z = z.repeat(num_points, 1)
    z[:, dim] = x.contiguous()
    outputs = torch.sigmoid(self.model.decode(z))
    interp = make_grid(outputs.cpu(), nrow=num_points, pad_value=1.0)
    return interp
